In [1]:
import benepar
benepar.download('benepar_en3_large')

[nltk_data] Downloading package benepar_en3_large to
[nltk_data]     /Users/simonyu/nltk_data...
[nltk_data]   Package benepar_en3_large is already up-to-date!


True

In [2]:
import benepar, spacy
nlp = spacy.load('en_core_web_md')
if spacy.__version__.startswith('2'):
        nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
    nlp.add_pipe("benepar", config={"model": "benepar_en3"})
doc = nlp("The time for action is now. It's never too late to do something.")
sent = list(doc.sents)[0]
print(sent._.parse_string)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))


/Users/simonyu/opt/anaconda3/envs/robust/lib/python3.8/site-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


In [3]:
def find_root_verb_and_its_dobj(tree_root):
    # first check if the current node and its children satisfy the condition
    if tree_root.pos_ == "VERB":
        for child in tree_root.children:
            if child.dep_ == "dobj" and child.pos_ == "NOUN":
                return tree_root.lemma_, child.lemma_
        return tree_root.lemma_, None
    # if not, check its children
    for child in tree_root.children:
        return find_root_verb_and_its_dobj(child)
    # if no children satisfy the condition, return None
    return None, None

def find_root_verb_and_its_dobj_in_string(s):
    doc = nlp(s)
    first_sent = list(doc.sents)[0]
    return find_root_verb_and_its_dobj(first_sent.root)

find_root_verb_and_its_dobj_in_string("Write me a story about education.")

('write', 'story')

In [7]:
import pandas as pd
import json
import tqdm

generated_data_path = "../self-seq/data/alpaca_final/alpaca_llama70b_iteration_2_new.jsonl"
data_paths = [
    "../self-seq/data/flancot_full/flancot_100k-iteration_1-iter.jsonl",
    "../self-seq/data/flancot_full/flancot_100k-iteration_2-iter.jsonl"
]
machine_generated_tasks = []
for path in data_paths:
    with open(path, "r") as fin:
        for line in fin:
            machine_generated_tasks.append(json.loads(line))
print(len(machine_generated_tasks))

machine_generated_tasks = [task for task in machine_generated_tasks if task["option"] != "D"]
instructions = list(set([task["instruction"] for task in machine_generated_tasks]))
print(len(instructions))

# split by ". Then", "and then", "and", ". "
split_instructions = []
for instruction in instructions:
    split_instruction = instruction.split(". Then")
    split_instruction = [ins.split("and then") for ins in split_instruction]
    split_instruction = [ins.split("and") for ins in split_instruction]
    split_instruction = [ins.split(". ") for ins in split_instruction]
    split_instructions.append(split_instruction)
    
# raw_phrases = []
# for instruction in tqdm.tqdm(instructions[:500]):
#     try:
#         verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
#         raw_phrases.append({
#             "verb": verb,
#             "noun": noun,
#             "instruction": instruction
#         })
#     except Exception as e:
#         print(e)
#         # print(instruction)

200000
75488


In [8]:
len(list(set([(task["instruction"], task["option"]) for task in machine_generated_tasks])))

75488

In [9]:
raw_phrases = pd.DataFrame(raw_phrases)
phrases = pd.DataFrame(raw_phrases).dropna()
phrases[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)

verb        noun     
analyze     sentence     7
generate    list         4
identify    word         4
analyze     tone         4
brainstorm  list         4
                        ..
determine   structure    1
            sum          1
            topic        1
            type         1
write       word         1
Length: 284, dtype: int64

In [10]:
top_verbs = phrases[["verb"]].groupby(["verb"]).size().nlargest(20).reset_index()

df = phrases[phrases["verb"].isin(top_verbs["verb"].tolist())]
# df = df[~df["noun"].isin(["I", "what"])]
# df = phrases
# df[~df["verb"].isin(top_verbs["verb"].tolist())]["verb"] = "other"
# df[~df["verb"].isin(top_verbs["verb"].tolist())]["noun"] = "other"
df = df.groupby(["verb", "noun"]).size().reset_index().rename(columns={0: "count"}).sort_values(by=["count"], ascending=False)
# df = df[df["count"] > 10]
df = df.groupby("verb").apply(lambda x: x.sort_values("count", ascending=False).head(4)).reset_index(drop=True)
df

,verb,noun,count
0,analyze,sentence,7
1,analyze,tone,4
2,analyze,poem,3
3,analyze,situation,3
4,brainstorm,list,4
...,...,...,...
74,review,context,1
75,write,function,3
76,write,paragraph,1
77,write,code,1


In [11]:
import plotly.graph_objects as go
import plotly.express as px

# df["blank"] = "ROOT"
# df = phrases.groupby(["verb", "noun"]).size().sort_values(ascending=False).head(5).reset_index().rename(columns={0: "count"})

# df = df[df["count"] > 30]
fig = px.sunburst(df, path=['verb', 'noun'], values='count')
# fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
    font_family="Times New Roman",
)
fig.show()
# fig.write_html("output/verb_noun.html")
# fig.savefig("output/verb_noun.pdf")
     